# PGIP Hands-On Walkthrough
This notebook guides you through running the PanGenome Insight Platform (PGIP) locally, interacting with the REST API, and registering plugins backed by the sample datasets bundled with the repository.

## 1. Prerequisites
* Python 3.11 with `virtualenv` or `conda`.
* Node.js 18+ for the React frontend.
* The repository cloned to your machine.
* (Optional) `httpie` or `curl` for manual API calls.

## 2. Start the Backend
Run these commands from the repository root to install dependencies, apply migrations, and launch the FastAPI server:
```powershell
python -m pip install -r backend/requirements.txt
alembic -c backend\alembic.ini upgrade head
uvicorn app.main:app --reload --port 8000
```
The backend seeds two plugin manifests from the JSON files in `data/plugins/`, which reference the example VCF and GFA assets located under `data/`.

## 3. Inspect the Plugin Registry via Python
In a new terminal, activate the same virtual environment and run the following code cell to query the live API. Update `API_URL` if you run PGIP on a different host or port.

In [ ]:
import requests
from pprint import pprint

API_URL = "http://127.0.0.1:8000"
plugins = requests.get(f"{API_URL}/api/v1/plugins/").json()
print(f"Loaded {len(plugins)} plugins")
pprint(plugins)

## 4. Register a New Plugin
The next cell loads one of the sample manifests, tweaks the version number, and posts it back to the API.

In [ ]:
import json
from pathlib import Path

manifest_path = Path("data/plugins/frequency-aggregator.json")
manifest = json.loads(manifest_path.read_text())
manifest["version"] = "0.1.1-dev"  # keep history intact
response = requests.post(f"{API_URL}/api/v1/plugins/", json=manifest)
response.raise_for_status()
response.json()

## 5. Explore the Frontend
Launch the React app in another terminal:
```powershell
cd frontend
npm install
npm run dev -- --host
```
Open the printed URL (default http://127.0.0.1:5173) to view the dashboard, filter plugins, and submit manifests through the UI form.

## 6. Run the CLI
The Typer CLI mirrors the REST endpoints. From the repository root:
```powershell
cd cli
python -m pgip_cli.app plugins list --api-url http://127.0.0.1:8000
python -m pgip_cli.app plugins stats --api-url http://127.0.0.1:8000
```

## 8. Next Experiments

* Swap the sample VCF (`data/variants/example.vcf`) with a different dataset and re-run the loader.
* Extend a manifest with new tags or resources and confirm the UI updates.
* Integrate a Nextflow pipeline run that produces outputs consumed by a plugin.


## 7. Run the Ingest Pipeline

- Install [Nextflow](https://www.nextflow.io/) (requires Java 11+) and ensure the `nextflow` command is on your PATH or note its location for the CLI option `--nextflow-bin`.
- Start the backend (`uvicorn app.main:app --reload --port 8000`) so the ingestion endpoint is reachable.
- From the repository root, execute the bundled workflow with the CLI wrapper:

```powershell
cd cli
python -m pgip_cli.app pipelines run --backend-api http://127.0.0.1:8000
```

- When the run finishes, inspect the outputs in `results/ingest/` and hit the FastAPI endpoint (or the CLI helper) to confirm the ingest summary:

```powershell
python -m pgip_cli.app pipelines list
Invoke-RestMethod -Method Get -Uri http://127.0.0.1:8000/api/v1/assets/vcf | ConvertTo-Json -Depth 4
```

The CLI streams the underlying `nextflow run workflows/nextflow/ingest_pangenome.nf` invocation and reports any non-zero exit status.
